### Creation of the Family Layer

In [244]:
import pandas as pd
from itertools import combinations
import numpy as np
pd.options.mode.chained_assignment = None

In [232]:
## read in the data

##i could merge on famID 


registry_data_one_year = pd.read_csv("testdata.csv")

In [233]:
registry_data_one_year

,Unnamed: 0,Sun2000niva_old,SUN2000Niva,SUN2000Inr,SUN2000Grp,Barn0_3,Barn4_6,Barn7_10,Barn11_15,Barn16_17,Barn18plus,Barn18_19,Barn20plus,PersonNr,famID,FodesleAr,Alder
0,0,15895,300,761d,Gymnasial utbildning med inriktning mot barn o...,0,0,0,0,0,0,0,0,19370429-1669,19350916-1755,1937,53
1,1,81607,5,840d,Transportutbildning på eftergymnasial nivå ...,0,0,1,1,1,0,0,0,19380301-8616,19350916-1755,1938,52
2,2,70251,300,811d,Hotell- och restaurangutbildning på gymnasial ...,1,2,0,2,2,2,0,0,19350916-1755,19350916-1755,1935,55
3,3,20859,5,762a,Social service- och omsorgsutbildning ...,2,0,2,1,0,0,0,0,19311224-1125,19311224-1125,1931,58
4,4,15015,532,106k,Minst 30 högskolepoäng inom lant- och skogsbru...,0,0,0,0,0,0,0,0,19530115-9550,19311224-1125,1953,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,51220,522,101c,Minst 30 högskolepoäng inom pedagogik och lära...,1,1,1,2,0,0,0,0,19321018-9211,19321018-9211,1932,58
9996,9996,59510,531,312c,Samhälls- och beteendevetenskaplig högskoleutb...,1,0,0,1,0,3,0,0,19290710-2723,19010428-8159,1929,61
9997,9997,79185,557,724c,Tandläkarutbildning ...,0,0,2,0,1,0,0,0,19010428-8159,19010428-8159,1901,89
9998,9998,72465,557,146f,Yrkeslärarutbildning ...,0,0,0,1,0,0,0,0,19390318-1701,19300628-6725,1939,51


In [234]:
#basic step --> connect everyone in the same family:
# Iterate through unique values in 'ConnectionColumn'


#
connections = pd.DataFrame()
for family in registry_data_one_year['famID'].unique():
    # Filter rows where 'famID' has the current value
    subset = registry_data_one_year[registry_data_one_year['famID'] == family]
    
    # Create pairs of IDs
    pairs = pd.merge(subset[['PersonNr']], subset[['PersonNr']], how='cross')
    
    # Add the connection type column
    pairs['ConnectionType'] = "family"
    
    # Append to the connections DataFrame
    connections = pd.concat([connections, pairs])

In [235]:
connections

,PersonNr_x,PersonNr_y,ConnectionType
0,19370429-1669,19370429-1669,family
1,19370429-1669,19380301-8616,family
2,19370429-1669,19350916-1755,family
3,19380301-8616,19370429-1669,family
4,19380301-8616,19380301-8616,family
...,...,...,...
3,19010428-8159,19010428-8159,family
0,19390318-1701,19390318-1701,family
1,19390318-1701,19300628-6725,family
2,19300628-6725,19390318-1701,family


In [379]:
def find_parent_child_relationship(subset):
    parent_relationships = []
    parents = subset[subset['Barn18plus'] > 0]
    children = subset[subset['Alder'] >= 18]
    
    #to create siblings
    dfs_to_concat = []
    for _, parent in parents.iterrows():
            age_difference = parent["Alder"] - children["Alder"]
            eligible_children = children[age_difference > 15]
            if not eligible_children.empty:
            
            
            # Assuming "Kön" is a column indicating gender
            #gender = 'mother' if parent['Kön'] == 'female' else 'father'
            
                # Create DataFrame for each parent-child relationship
                child_data = {'personNr1': eligible_children["PersonNr"].tolist(),
                              'personNr2': parent["PersonNr"],
                              'connection': "child of"}

                parent_data = {'personNr1': parent["PersonNr"],
                               'personNr2': eligible_children["PersonNr"].tolist(),
                               'connection': "parent of"}

                parent_relationships.extend([pd.DataFrame(child_data), pd.DataFrame(parent_data)])
                #print(eligible_children)
                if len(eligible_children)>1:
                    personNr2_list = eligible_children['PersonNr'].tolist()
                    personNr2_list = sorted(personNr2_list)
                    print(personNr2_list)
                    print(len(personNr2_list))
                    if len(personNr2_list) == 2:
                        # If length is 2, create a single row with 'siblings' as the connection
                        row_data = pd.DataFrame([{'personNr1': personNr2_list[0], 'personNr2': personNr2_list[1], 'connection': 'siblings'}])
                        dfs_to_concat.extend([pd.DataFrame(row_data)])
                        
                    elif len(personNr2_list) > 2:
                        # If length is greater than 2, create rows for all combinations
                        for pair in combinations(personNr2_list, 2):
                            row_data = pd.DataFrame([{'personNr1': pair[0], 'personNr2': pair[1], 'connection': 'siblings'}])
                            dfs_to_concat.extend([pd.DataFrame(row_data)])
                   
                

    if parent_relationships:
        # Concatenate all DataFrames in the list
        parent_relationships = pd.concat(parent_relationships, ignore_index=True)
        if dfs_to_concat:
            print(dfs_to_concat)
            dfs_to_concat = pd.DataFrame(pd.concat(dfs_to_concat).drop_duplicates(ignore_index=True))
            print(dfs_to_concat)
            parent_relationships = pd.concat([parent_relationships, dfs_to_concat], ignore_index=True)
    else:
        # Create an empty DataFrame
        parent_relationships = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])

            
    return parent_relationships


In [382]:
def find_relationships(subset, current_connections):
    #should i change approach here? 
    couple_relationships = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    selected = subset[['PersonNr', 'Alder']]
    for i in range(len(selected)-1):
        for j in range(i+1, len(selected)):

            person1 = selected.iat[i, 0]
            person2 = selected.iat[j, 0]
            # Calculate age difference
            age_difference = abs(selected.iat[i, 1] - selected.iat[j, 1])
        
            # Check if age difference is less than or equal to 15 years
            
            siblings_possible = (((current_connections['personNr1'] != person1) & (current_connections['personNr2'] != person2) & (current_connections['connection'] != "siblings")) | ((current_connections['personNr1'] != person2) & (current_connections['personNr2'] != person1) & (current_connections['connection'] != "siblings")) ).all()
            #add check for amount of children (?)
            if age_difference <= 13 and siblings_possible != True :
                row_data = pd.DataFrame([{'personNr1':person1 , 'personNr2': person2, 'connection': "partner of"}])
                couple_relationships = pd.concat([couple_relationships, row_data])
               
    return couple_relationships

In [294]:
num_rows = 30

# Create a DataFrame with random numbers in 'personnr1' and 'personnr2'
test_case_grandparent = pd.DataFrame({
    'personNr1': np.random.randint(11, 40, size=num_rows),
    'personNr2': np.random.randint(0, 33, size=num_rows),
})

# Introduce some matches between 'personnr1' and 'personnr2'
num_matches = 3
match_indices = np.random.choice(num_rows, size=num_matches, replace=False)
test_case_grandparent.loc[match_indices, 'personNr2'] = test_case_grandparent.loc[match_indices, 'personNr1']

# Create a 'connection' column with values 'parent of' or 'sibling'
test_case_grandparent['connection'] = np.random.choice(['parent of'], size=num_rows)
test_case_grandparent = test_case_grandparent[test_case_grandparent['personNr1'] != test_case_grandparent['personNr2']]


outcome_2 = find_cousines(test_case_grandparent)
print(outcome_2)

Empty DataFrame
Columns: [personNr1, personNr2, connection]
Index: []


In [352]:
def find_siblings(family_connections):
        # Assuming connections is your DataFrame
    selection = family_connections[family_connections["connection"] == "parent of"]

    # Find all the siblings (people with the same parent)
    result = selection.groupby('personNr1')['personNr2'].agg(list).reset_index()
    result_filtered = result[result['personNr2'].apply(len) > 1]
    
    # Create a list to store DataFrames
    dfs_to_concat = []

    for _, row in result_filtered.iterrows():
        personNr2_list = row['personNr2']
        personNr2_list = sorted(personNr2_list)
        # Check the length of the list
        if len(personNr2_list) == 2:
            # If length is 2, create a single row with 'siblings' as the connection
            row_data = pd.DataFrame([{'personNr1': personNr2_list[0], 'personNr2': personNr2_list[1], 'connection': 'siblings'}])
            dfs_to_concat.append(row_data)
        elif len(personNr2_list) > 2:
            # If length is greater than 2, create rows for all combinations
            for pair in combinations(personNr2_list, 2):
                row_data = pd.DataFrame([{'personNr1': pair[0], 'personNr2': pair[1], 'connection': 'siblings'}])
                dfs_to_concat.append(row_data)
    
    if dfs_to_concat:
        dfs_to_concat = pd.concat(dfs_to_concat).drop_duplicates(ignore_index=True)
    
    
    return dfs_to_concat

In [341]:
def find_grandparents_aunts(family_connections):
    
    #make this without warnings --> change append to concat 
    
    grandparents_list = []
    parent_rows = family_connections[family_connections['connection'] == 'parent of']
    sibling_rows = family_connections[family_connections['connection'] == 'siblings']
    for _, parent_row in parent_rows.iterrows():
        # Find rows where 'Personnr1' in the original dataframe matches 'personNr2' in the current row
        matching_rows = parent_rows[parent_rows['personNr2'] == parent_row['personNr1']]
        
        matching_rows_auntsuncles = sibling_rows[(sibling_rows['personNr2'] == parent_row['personNr1']) |(sibling_rows['personNr1'] == parent_row['personNr1'])]
        # Iterate through matching rows and add to the result dataframe
        if not matching_rows.empty:
            for _, matching_row in matching_rows.iterrows():

                grandparents_list.append({
                    'personNr1': matching_row['personNr1'],
                    'personNr2': parent_row['personNr2'],
                    'connection': "grandparent of"
                })
                grandparents_list.append({
                    'personNr1': parent_row['personNr2'],
                    'personNr2': matching_row['personNr1'],
                    'connection': "grandchild of"
                })
        
        
        if not matching_rows_auntsuncles.empty:
            for _, matching_row in matching_rows_auntsuncles.iterrows():

        #aunts and uncles
                grandparents_list.append({
                    'personNr1': matching_row['personNr1'],
                    'personNr2': parent_row['personNr2'],
                    'connection': "aunt/uncle of"
                })
                grandparents_list.append({
                    'personNr1': parent_row['personNr2'],
                    'personNr2': matching_row['personNr1'],
                    'connection': "niece/newphew of"
                })

    return   pd.DataFrame(grandparents_list, columns=['personNr1', 'personNr2', 'connection'])
            
    

In [347]:
def find_cousines(family_connections):
    cousines= pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    aunt_uncle_rows = family_connections[family_connections['connection'] == 'aunt/uncle of']
    parent_rows = family_connections[family_connections['connection'] == 'parent of']
    
    aunt_uncle_rows = aunt_uncle_rows.reset_index(drop=True)
    parent_rows = parent_rows.reset_index(drop=True)
    for _, aunt_uncle_row in aunt_uncle_rows.iterrows():
        matching_rows = parent_rows[parent_rows['personNr1'] == aunt_uncle_row['personNr1']]
        
        for _, matching_row in matching_rows.iterrows():
            cousines = cousines.append({
                'personNr1': matching_row['personNr2'],
                'personNr2': aunt_uncle_row['personNr2'],
                'connection': "cousins"
            }, ignore_index=True)
            
    return cousines

In [383]:
parents = pd.DataFrame()
registry_100 = registry_data_one_year.head(100)



#next step
#create functions to call for creation of children
#create function to call for creation of partners that work for the subset that append to a dictionary(?)

#create dicitionaries and then create a final dataframe for efficiency --> overwork all the steps so far
def create_family_layer(registry_data):
    connections = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    for family in registry_data['famID'].unique():
        # Filter rows where 'famID' has the current value
        subset = registry_data[registry_data['famID'] == family]
        # Check if each row has a value greater than or equal to 0 in the 'your_column' column
        if len(subset)> 1: 
            #will there be an option to check which year, we are working on

            #work on child-parent relationship 
            #parents = subset[(subset['Barn18plus'] > 0) | (subset['Barn16_17'] >0 ) | (subset['Barn18_19'] > 0) | (subset['Barn20plus'] >0 )].any(axis=1)
            parents = subset[(subset['Barn18plus']>0) |  (subset['Barn16_17'] >0 )  | (subset['Barn18_19'] > 0) | (subset['Barn20plus'] >0 )]
            #print(parents)
            if len(parents)> 1:
                output = find_parent_child_relationship(subset)

                connections = pd.concat([connections, output])

            #work on the partner relationship
            relationships = find_relationships(subset, connections)
            connections = pd.concat([connections, relationships])
            
            
        #assumption --> partners have minimum one child of the same age --> double check with famId creation about partners
        # also connected if both have no children (marriage?)
        
    #dfs_to_concat = find_siblings(connections)
    #if not dfs_to_concat.empty:
    #    connections = pd.concat([connections] + [dfs_to_concat], ignore_index=True)

    #grandparent/ aunts/uncles / niece .. /cousin relationship

    
    grandparents_connections = find_grandparents_aunts(connections)
    if not grandparents_connections.empty:
        connections = pd.concat([connections] + [grandparents_connections], ignore_index=True)


    cousine_connections = find_cousines(connections)
    if not cousine_connections.empty:
        connections = pd.concat([connections] + [cousine_connections], ignore_index=True)
    
    
    return connections

network = create_family_layer(registry_data_one_year)

['19240104-4206', '19241106-5535']
2
[       personNr1      personNr2 connection
0  19240104-4206  19241106-5535   siblings]
       personNr1      personNr2 connection
0  19240104-4206  19241106-5535   siblings
['19631119-6217', '19700304-4935']
2
[       personNr1      personNr2 connection
0  19631119-6217  19700304-4935   siblings]
       personNr1      personNr2 connection
0  19631119-6217  19700304-4935   siblings
['19600821-5688', '19710719-4234']
2
[       personNr1      personNr2 connection
0  19600821-5688  19710719-4234   siblings]
       personNr1      personNr2 connection
0  19600821-5688  19710719-4234   siblings
['19290701-9307', '19660531-9640']
2
[       personNr1      personNr2 connection
0  19290701-9307  19660531-9640   siblings]
       personNr1      personNr2 connection
0  19290701-9307  19660531-9640   siblings
['19440915-7995', '19481121-1745', '19660610-1198']
3
[       personNr1      personNr2 connection
0  19440915-7995  19481121-1745   siblings,        personN

['19340521-5451', '19540311-5753', '19650528-1334']
3
[       personNr1      personNr2 connection
0  19340521-5451  19540311-5753   siblings,        personNr1      personNr2 connection
0  19340521-5451  19650528-1334   siblings,        personNr1      personNr2 connection
0  19540311-5753  19650528-1334   siblings]
       personNr1      personNr2 connection
0  19340521-5451  19540311-5753   siblings
1  19340521-5451  19650528-1334   siblings
2  19540311-5753  19650528-1334   siblings
['19240905-3146', '19440508-7645']
2
[       personNr1      personNr2 connection
0  19240905-3146  19440508-7645   siblings]
       personNr1      personNr2 connection
0  19240905-3146  19440508-7645   siblings
['19320929-3124', '19460827-3347']
2
[       personNr1      personNr2 connection
0  19320929-3124  19460827-3347   siblings]
       personNr1      personNr2 connection
0  19320929-3124  19460827-3347   siblings
['19530327-3592', '19600716-8448']
2
[       personNr1      personNr2 connection
0  195303

['19520711-5350', '19610428-6365']
2
['19520711-5350', '19610428-6365']
2
[       personNr1      personNr2 connection
0  19520711-5350  19610428-6365   siblings,        personNr1      personNr2 connection
0  19520711-5350  19610428-6365   siblings]
       personNr1      personNr2 connection
0  19520711-5350  19610428-6365   siblings
['19250222-5632', '19260422-5767', '19381101-3621']
3
[       personNr1      personNr2 connection
0  19250222-5632  19260422-5767   siblings,        personNr1      personNr2 connection
0  19250222-5632  19381101-3621   siblings,        personNr1      personNr2 connection
0  19260422-5767  19381101-3621   siblings]
       personNr1      personNr2 connection
0  19250222-5632  19260422-5767   siblings
1  19250222-5632  19381101-3621   siblings
2  19260422-5767  19381101-3621   siblings
['19350802-5200', '19530814-3499', '19630703-2848']
3
[       personNr1      personNr2 connection
0  19350802-5200  19530814-3499   siblings,        personNr1      personNr2 con

['19311020-7804', '19700127-2104']
2
[       personNr1      personNr2 connection
0  19311020-7804  19700127-2104   siblings]
       personNr1      personNr2 connection
0  19311020-7804  19700127-2104   siblings
['19500208-3577', '19670806-8658']
2
[       personNr1      personNr2 connection
0  19500208-3577  19670806-8658   siblings]
       personNr1      personNr2 connection
0  19500208-3577  19670806-8658   siblings
['19590508-9286', '19650217-5430']
2
[       personNr1      personNr2 connection
0  19590508-9286  19650217-5430   siblings]
       personNr1      personNr2 connection
0  19590508-9286  19650217-5430   siblings
['19340814-3565', '19640522-6657']
2
[       personNr1      personNr2 connection
0  19340814-3565  19640522-6657   siblings]
       personNr1      personNr2 connection
0  19340814-3565  19640522-6657   siblings
['19590301-8546', '19651030-1454']
2
[       personNr1      personNr2 connection
0  19590301-8546  19651030-1454   siblings]
       personNr1      personNr2

['19631031-9452', '19680210-5112']
2
[       personNr1      personNr2 connection
0  19631031-9452  19680210-5112   siblings]
       personNr1      personNr2 connection
0  19631031-9452  19680210-5112   siblings
['19461211-4675', '19581023-7576']
2
['19461211-4675', '19581023-7576']
2
[       personNr1      personNr2 connection
0  19461211-4675  19581023-7576   siblings,        personNr1      personNr2 connection
0  19461211-4675  19581023-7576   siblings]
       personNr1      personNr2 connection
0  19461211-4675  19581023-7576   siblings
['19260407-4887', '19310106-6501', '19590824-1185']
3
[       personNr1      personNr2 connection
0  19260407-4887  19310106-6501   siblings,        personNr1      personNr2 connection
0  19260407-4887  19590824-1185   siblings,        personNr1      personNr2 connection
0  19310106-6501  19590824-1185   siblings]
       personNr1      personNr2 connection
0  19260407-4887  19310106-6501   siblings
1  19260407-4887  19590824-1185   siblings
2  193101

['19531019-4982', '19570207-7489', '19620207-1858']
3
[       personNr1      personNr2 connection
0  19531019-4982  19570207-7489   siblings,        personNr1      personNr2 connection
0  19531019-4982  19620207-1858   siblings,        personNr1      personNr2 connection
0  19570207-7489  19620207-1858   siblings]
       personNr1      personNr2 connection
0  19531019-4982  19570207-7489   siblings
1  19531019-4982  19620207-1858   siblings
2  19570207-7489  19620207-1858   siblings
['19490130-7954', '19570802-3341', '19700104-7510']
3
[       personNr1      personNr2 connection
0  19490130-7954  19570802-3341   siblings,        personNr1      personNr2 connection
0  19490130-7954  19700104-7510   siblings,        personNr1      personNr2 connection
0  19570802-3341  19700104-7510   siblings]
       personNr1      personNr2 connection
0  19490130-7954  19570802-3341   siblings
1  19490130-7954  19700104-7510   siblings
2  19570802-3341  19700104-7510   siblings
['19590309-5577', '19620

['19631103-1848', '19671222-3986']
2
[       personNr1      personNr2 connection
0  19631103-1848  19671222-3986   siblings]
       personNr1      personNr2 connection
0  19631103-1848  19671222-3986   siblings
['19241201-7754', '19500113-6748']
2
[       personNr1      personNr2 connection
0  19241201-7754  19500113-6748   siblings]
       personNr1      personNr2 connection
0  19241201-7754  19500113-6748   siblings
['19491126-4058', '19581123-8758']
2
[       personNr1      personNr2 connection
0  19491126-4058  19581123-8758   siblings]
       personNr1      personNr2 connection
0  19491126-4058  19581123-8758   siblings
['19390913-5079', '19501110-7994', '19670423-1415']
3
[       personNr1      personNr2 connection
0  19390913-5079  19501110-7994   siblings,        personNr1      personNr2 connection
0  19390913-5079  19670423-1415   siblings,        personNr1      personNr2 connection
0  19501110-7994  19670423-1415   siblings]
       personNr1      personNr2 connection
0  19390

['19510529-2590', '19671223-1998']
2
[       personNr1      personNr2 connection
0  19510529-2590  19671223-1998   siblings]
       personNr1      personNr2 connection
0  19510529-2590  19671223-1998   siblings
['19250730-8131', '19390609-1524']
2
[       personNr1      personNr2 connection
0  19250730-8131  19390609-1524   siblings]
       personNr1      personNr2 connection
0  19250730-8131  19390609-1524   siblings
['19460731-1000', '19580906-6905', '19610727-3760']
3
[       personNr1      personNr2 connection
0  19460731-1000  19580906-6905   siblings,        personNr1      personNr2 connection
0  19460731-1000  19610727-3760   siblings,        personNr1      personNr2 connection
0  19580906-6905  19610727-3760   siblings]
       personNr1      personNr2 connection
0  19460731-1000  19580906-6905   siblings
1  19460731-1000  19610727-3760   siblings
2  19580906-6905  19610727-3760   siblings
['19300301-6545', '19580808-9797']
2
[       personNr1      personNr2 connection
0  193003

['19381207-1696', '19610322-6222']
2
[       personNr1      personNr2 connection
0  19381207-1696  19610322-6222   siblings]
       personNr1      personNr2 connection
0  19381207-1696  19610322-6222   siblings
['19530516-9182', '19580511-3508']
2
[       personNr1      personNr2 connection
0  19530516-9182  19580511-3508   siblings]
       personNr1      personNr2 connection
0  19530516-9182  19580511-3508   siblings
['19510723-8936', '19630224-2584']
2
[       personNr1      personNr2 connection
0  19510723-8936  19630224-2584   siblings]
       personNr1      personNr2 connection
0  19510723-8936  19630224-2584   siblings
['19650503-8656', '19710826-1503']
2
[       personNr1      personNr2 connection
0  19650503-8656  19710826-1503   siblings]
       personNr1      personNr2 connection
0  19650503-8656  19710826-1503   siblings
['19540224-5672', '19700925-7582']
2
['19540224-5672', '19700925-7582']
2
[       personNr1      personNr2 connection
0  19540224-5672  19700925-7582   sibl

C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\1858042313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cousines = cousines.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\1858042313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cousines = cousines.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\1858042313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cousines = cousines.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\1858042313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cousines = cousines.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\1858042313.py:12: FutureWarning:

In [384]:
network

,personNr1,personNr2,connection
0,19241106-5535,19080605-8105,child of
1,19240104-4206,19080605-8105,child of
2,19080605-8105,19241106-5535,parent of
3,19080605-8105,19240104-4206,parent of
4,19240104-4206,19241106-5535,siblings
...,...,...,...
5761,19650907-3708,19590423-5699,cousins
5762,19590423-5699,19590423-5699,cousins
5763,19621002-8325,19621002-8325,cousins
5764,19550515-3421,19550515-3421,cousins


In [315]:
#basic analysis of network 

In [47]:
def find_siblings(family_connections):
    unique_combinations_df = pd.DataFrame()
    selection = family_connections[family_connections["connection"] == "parent of"]
    siblings = selection.merge(selection, how='left', on='personNr1')
    # Find all the siblings (people with the same parent)

    siblings_df = siblings[siblings['personNr2_x'] != siblings['personNr2_y']][['personNr2_x', 'personNr2_y']]

    #unique_combinations_df = siblings_df.drop_duplicates(subset=['personNr2_x', 'personNr2_y'])
    # Convert columns to tuples and drop duplicates
    siblings_df['tuple_key'] = siblings_df[['personNr2_x', 'personNr2_y']].apply(lambda x: tuple(sorted(x)), axis=1)
    unique_combinations_df = siblings_df.drop_duplicates(subset='tuple_key')

    # Drop the temporary column
    unique_combinations_df = unique_combinations_df.drop(columns=['tuple_key'])
    unique_combinations_df['connection'] = 'siblings'

    # Rename columns by removing the suffixes _x and _y
    unique_combinations_df.columns = unique_combinations_df.columns.str.replace('_x', '').str.replace('_y', '')
    
    
    return unique_combinations_df

In [82]:
def find_grandparents(family_connections):
    output = pd.DataFrame()
    selection = family_connections[family_connections["connection"] == "parent of"]
    
    grandparents_options = selection.merge(selection, left_on='personNr1', right_on='personNr2', suffixes=('_childside', '_grandparentside'))
    
    grandparents = grandparents_options[["personNr1_grandparentside", "personNr2_childside" ]]
    #grandparents = grandparents.columns.str.replace('_grandparentside', '').str.replace('_childside', '')
    
    grandparents.columns = ['personNr1', 'personNr2']

    # Add a new column 'connection' with the value 'grandparent of'
    grandparents['connection'] = 'grandparent of'
    
    grandchildren =  grandparents_options[["personNr2_childside", "personNr1_grandparentside" ]]
    grandchildren.columns = ['personNr1', 'personNr2']
    grandchildren['connection'] = 'grandchild of'
    
    output = pd.concat([grandchildren] + [grandparents], ignore_index=True)
    
    
    return output

In [111]:
def find_aunts_uncles(family_connections):
    output1 = pd.DataFrame()
    output2= pd.DataFrame()
    selection_parent = family_connections[family_connections["connection"] == "parent of"]
    selection_sibling = family_connections[family_connections["connection"] == "siblings"]
    
    options1 = selection_parent.merge(selection_sibling, left_on="personNr1", right_on="personNr1", suffixes=('_parent', '_aunt'))
    options2 = selection_parent.merge(selection_sibling, left_on="personNr1", right_on="personNr2", suffixes=('_parent', '_aunt'))
    if not options1.empty:
        aunt_connections = options1[["personNr2_aunt", "personNr2_parent" ]]
        niece_connections = options1[["personNr2_parent", "personNr2_aunt"]]
        #grandparents = grandparents.columns.str.replace('_grandparentside', '').str.replace('_childside', '')
    
        aunt_connections.columns = ['personNr1', 'personNr2']
        niece_connections.columns = ['personNr1', 'personNr2']
        aunt_connections = aunt_connections[aunt_connections['personNr1'] != aunt_connections['personNr2']]
        niece_connections = niece_connections[niece_connections['personNr1'] != niece_connections['personNr2']]

        aunt_connections['connection'] = 'aunt/uncle of'
        niece_connections['connection'] = 'niece/newphew of'
        output1 = pd.concat([aunt_connections] + [niece_connections], ignore_index=True)
    if not options2.empty: 
        aunt_connections = options2[["personNr1_aunt", "personNr2_parent" ]]
        niece_connections = options2[["personNr2_parent", "personNr1_aunt"]]
        #grandparents = grandparents.columns.str.replace('_grandparentside', '').str.replace('_childside', '')
    
        aunt_connections.columns = ['personNr1', 'personNr2']
        niece_connections.columns = ['personNr1', 'personNr2']
        aunt_connections = aunt_connections[aunt_connections['personNr1'] != aunt_connections['personNr2']]
        niece_connections = niece_connections[niece_connections['personNr1'] != niece_connections['personNr2']]


        aunt_connections['connection'] = 'aunt/uncle of'
        niece_connections['connection'] = 'niece/newphew of'
        output2 = pd.concat([aunt_connections] + [niece_connections], ignore_index=True)
    final_output = pd.concat([output1] + [output2], ignore_index=True)
    return final_output

In [420]:
family_connections_data = pd.DataFrame({
    'personNr1': [2, 2, 4],
    'personNr2' : [5,3,7,],
    'connection': ["aunt/uncle of", "parent of", "parent of"]
})

In [423]:
def find_cousins(family_connections):
    #cousines= pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    output2= pd.DataFrame()
    selection_aunts = family_connections[family_connections["connection"] == "aunt/uncle of"]
    selection_parents = family_connections[family_connections["connection"] == "parent of"]
    
    options = selection_aunts.merge(selection_parents, left_on="personNr1", right_on="personNr1", suffixes=('_aunt', '_parent'))
    cousin_connection=  options[["personNr2_aunt", "personNr2_parent" ]]
    
    cousin_connection['tuple_key'] = cousin_connection[["personNr2_aunt", "personNr2_parent"]].apply(lambda x: tuple(sorted(x)), axis=1)
    cousin_connection = cousin_connection.drop_duplicates(subset='tuple_key')

    cousin_connection = pd.DataFrame(cousin_connection['tuple_key'].tolist(), columns=['personNr1', 'personNr2'])
    #cousin_connection.columns = ['personNr1', 'personNr2']
    cousin_connection['connection'] = 'cousins'

    return cousin_connection

In [424]:
print(find_cousins(family_connections_data))

   personNr1  personNr2 connection
0          3          5    cousins


In [315]:
def find_relationships(subset, current_connections):

    couple_relationships = []
    selected = subset[['PersonNr', 'Alder']]
    merged_list = [(x, y) for x in selected['PersonNr'] for y in selected['PersonNr'] if x != y]

    result_list = [
    (
        x,
        y,
        abs(
            selected.loc[selected['PersonNr'] == x, 'Alder'].iloc[0]
            - selected.loc[selected['PersonNr'] == y, 'Alder'].iloc[0]
        ),
    )
    for x, y in merged_list
    if x != y
]

    filtered_list = [(min(x, y), max(x, y)) for x, y, age_diff in result_list if age_diff <= 13]

    unique_combinations_set = set(filtered_list)

    result_dict = {
    'personNr1': [x[0] for x in unique_combinations_set],
    'personNr2': [x[1] for x in unique_combinations_set],
}

    # Filter out the partners that are actually siblings
    if current_connections:
        connections_set = {(row['personNr1'], row['personNr2']) for _, row in current_connections.iterrows() if row['connection'] == 'siblings'}

        filtered_combinations_list = [
        {'personNr1': pair[0], 'personNr2': pair[1]} for pair in zip(result_dict['personNr1'], result_dict['personNr2']) if pair not in connections_set
    ]
        print(filtered_combinations_list)
    couple_relationships = unique_combinations_list
    return unique_combinations_list

In [320]:
##need data for this 

df_data = pd.DataFrame({
    'PersonNr': [2, 3, 4, 5],
    'Alder': [45, 42, 16, 18]
})


connections = [
    {'personNr1': 2, 'personNr2': 5, 'connection': 'parent of'},
    {'personNr1': 2, 'personNr2': 3, 'connection': 'siblings'},
    {'personNr1': 4, 'personNr2': 2, 'connection': 'parent of'}
]

print(connections)

#take care of order and check for sibling

[{'personNr1': 2, 'personNr2': 5, 'connection': 'parent of'}, {'personNr1': 2, 'personNr2': 3, 'connection': 'siblings'}, {'personNr1': 4, 'personNr2': 2, 'connection': 'parent of'}]


In [321]:
print(find_relationships(df_data, connections))

AttributeError: 'list' object has no attribute 'iterrows'

In [383]:
def find_siblings(family_connections):
    unique_combinations_df = pd.DataFrame()
    selection = family_connections[family_connections["connection"] == "parent of"]
    siblings = selection.merge(selection, how='left', on='personNr1')
    # Find all the siblings (people with the same parent)

    siblings_df = siblings[siblings['personNr2_x'] != siblings['personNr2_y']][['personNr2_x', 'personNr2_y']]

    #unique_combinations_df = siblings_df.drop_duplicates(subset=['personNr2_x', 'personNr2_y'])
    # Convert columns to tuples and drop duplicates
    siblings_df['tuple_key'] = siblings_df[['personNr2_x', 'personNr2_y']].apply(lambda x: tuple(sorted(x)), axis=1)
    unique_combinations_df = siblings_df.drop_duplicates(subset='tuple_key')

    # Drop the temporary column
    #unique_combinations_df = unique_combinations_df.drop(columns=['personNr2_x', 'personNr2_y'])
    #unique_combinations_df['connection'] = 'siblings'

    # Rename columns by removing the suffixes _x and _y
    #unique_combinations_df.columns = unique_combinations_df.columns.str.replace('2_x', '1').str.replace('_y', '')
    final_df = pd.DataFrame(unique_combinations_df['tuple_key'].tolist(), columns=['personNr1', 'personNr2'])

    # Add the 'connection' column
    final_df['connection'] = 'siblings'
    
    return final_df

In [390]:
family_connections_data = pd.DataFrame({
    'personNr1': ["223-2", "22-3", "22-3"],
    'personNr2' : ["322-2","19911004-7307","1995-1",],
    'connection': ["aunt/uncle of", "parent of", "parent of"]
})

In [391]:
print(find_siblings(family_connections_data))
outcome = find_siblings(family_connections_data)
print(outcome.dtypes)

       personNr1 personNr2 connection
0  19911004-7307    1995-1   siblings
personNr1     object
personNr2     object
connection    object
dtype: object


In [360]:
# Example DataFrame
data = {'personNr1': [2, 2, 4, 2],
        'personNr2': [1, 1, 5, 1],
        'connection': ['parent of', 'partner of', 'partner of', 'siblings']}
connections = pd.DataFrame(data)

# Filter the DataFrame
connections = connections[~((connections.duplicated(subset=['personNr1', 'personNr2'])) & (connections['connection'] == 'partner of'))]

# Display the result
print(connections)

   personNr1  personNr2  connection
0          2          1   parent of
2          4          5  partner of
3          2          1    siblings


In [417]:
def find_relationships(subset, current_connections):
    #should i change approach here? 
    #couple_relationships = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    couple_relationships = []
    selected = subset[['PersonNr', 'Alder']]
    selected = selected.sort_values(by="PersonNr")
    print(selected)
    if current_connections:
        current_connections = pd.DataFrame(current_connections, columns= ["personNr1", "personNr2", "connection"])
        connections_exist = True
    else:
        connections_exist = False


    for i in range(len(selected)-1):
        print(i)
        person1 = selected.iat[i, 0]
        print(person1)
        person2 = selected.iat[i+1, 0]
        print("person2")
        print(person2)
        # Calculate age difference
        age_difference = abs(selected.iat[i, 1] - selected.iat[i+1, 1])
        
            #checking if they might be siblings 

        if connections_exist == True:  
            siblings_possible = (((current_connections['personNr1'] == person1) & (current_connections['personNr2'] == person2) & (current_connections['connection'] == "siblings")) | ((current_connections['personNr1'] == person2) & (current_connections['personNr2'] == person1) & (current_connections['connection'] == "siblings")) ).any()
        else:
            siblings_possible=False
            #add check for amount of children (?)
        if age_difference <= 13 and siblings_possible==False:
                #row_data = pd.DataFrame([{'personNr1':person1 , 'personNr2': person2, 'connection': "partner of"}])
                #couple_relationships = pd.concat([couple_relationships, row_data])
            row_data = {'personNr1': person1, 'personNr2': person2, 'connection': "partner of"}
            couple_relationships.extend([row_data])

       
    return couple_relationships

In [419]:
data = {'personNr1': ["23-3", "222-3", "433-2"],
        'personNr2': ["133-2", "23-3", "533-2"],
        'connection': ['parent of', 'partner of', 'partner of']}

data1 = {'PersonNr': ["19911004-7307", "1995-1", "433-2"],
        'Alder': [18, 19, 60]}
connections = [{'personNr1': p1, 'personNr2': p2, 'connection': conn} for p1, p2, conn in zip(data['personNr1'], data['personNr2'], data['connection'])]
print(connections)
print(pd.DataFrame(connections))

print(pd.DataFrame(find_relationships(pd.DataFrame(data1), data)))

outcome1 = pd.DataFrame(find_relationships(pd.DataFrame(data1), data))

print(outcome.dtypes)

print(pd.concat([outcome, outcome1]))
filtered_df = pd.concat([outcome, outcome1])
print(filtered_df[(filtered_df.duplicated(subset=['personNr1', 'personNr2']))])

[{'personNr1': '23-3', 'personNr2': '133-2', 'connection': 'parent of'}, {'personNr1': '222-3', 'personNr2': '23-3', 'connection': 'partner of'}, {'personNr1': '433-2', 'personNr2': '533-2', 'connection': 'partner of'}]
  personNr1 personNr2  connection
0      23-3     133-2   parent of
1     222-3      23-3  partner of
2     433-2     533-2  partner of
        PersonNr  Alder
0  19911004-7307     18
1         1995-1     19
2          433-2     60
0
19911004-7307
person2
1995-1
1
1995-1
person2
433-2
       personNr1 personNr2  connection
0  19911004-7307    1995-1  partner of
        PersonNr  Alder
0  19911004-7307     18
1         1995-1     19
2          433-2     60
0
19911004-7307
person2
1995-1
1
1995-1
person2
433-2
personNr1     object
personNr2     object
connection    object
dtype: object
       personNr1 personNr2  connection
0  19911004-7307    1995-1    siblings
0  19911004-7307    1995-1  partner of
       personNr1 personNr2  connection
0  19911004-7307    1995-1  partn

In [397]:
def find_parent_child_relationship(subset, current_year):
    parent_relationships = []
    #also checking for 15, as some 15 year olds have been present in the data

    #probably a more elegant way to do this, but this is to make sure we match parent with a child in a right age group

    #could maybe be stored outside of this
    if current_year <= 2003:
        #parents_exist = len(subset[(subset['Barn18plus']>0) | (subset['Barn11_15']>0) |  (subset['Barn16_17'] >0 )  ]) > 0
        age_conditions_parents = [
        ('parents_18plus', subset['Barn18plus'] > 0),
        ('parents11_15', subset['Barn11_15'] > 0),
        ('parents16_17', subset['Barn16_17'] > 0),
        ]
        age_conditions_children = [
        ('children_18plus', subset['Alder'] >= 18),
        ('children11_15', subset['Alder'].between(11, 15)),
        ('children16_17', subset['Alder'].between(16, 17)),
        ]
        
    #add an else if here?
    else: 
        #parents_exist = len(subset[(subset['Barn11_15']>0) |  (subset['Barn16_17'] >0 )  | (subset['Barn18_19'] > 0) | (subset['Barn20plus'] >0 )]) > 0
        age_conditions_parents = [
        ('parents11_15', subset['Barn11_15'] > 0),
        ('parents16_17', subset['Barn16_17'] > 0),
        ('parents18_19', subset['Barn18_19'] > 0),
        ('parents_20plus', subset['Barn20plus'] > 0),
        ]
        #children = subset[subset['Alder'] >= 15]
        age_conditions_children = [
        ('children11_15', subset['Alder'].between(11, 15)),
        ('children16_17', subset['Alder'].between(16, 17)),
        ('children18_19', subset['Alder'].between(18, 19)),
        ('children_20plus', subset['Alder'] >= 20),  
        ]
    #children_list = []
    for (_, parent_condition), (_, children_condition) in zip(age_conditions_parents, age_conditions_children):

        parents = subset[parent_condition]
        children = subset[children_condition]

        if not children.empty and not parents.empty:
            for _, parent in parents.iterrows():
                age_difference = parent["Alder"] - children["Alder"]
                eligible_children = children[age_difference > 15]
                if not eligible_children.empty:
                    for _,child in eligible_children.iterrows():
                    # Assuming "Kön" is a column indicating gender
                    #gender = 'mother' if parent['Kön'] == 'female' else 'father'
                        
                        # Create DataFrame for each parent-child relationship
                        child_data = {'personNr1': child["PersonNr"],
                                    'personNr2': parent["PersonNr"],
                                'connection': "child of"}

                        parent_data = {'personNr1': parent["PersonNr"],
                                        'personNr2': child["PersonNr"],
                                        'connection': "parent of"}
                        parent_relationships.extend([child_data, parent_data])
                        
                        #parent_relationships.extend([pd.DataFrame(child_data), pd.DataFrame(parent_data)])
                            #print(eligible_children)
                    if len(eligible_children)>1:
                        personNr2_list = eligible_children['PersonNr'].tolist()
                        personNr2_list = sorted(personNr2_list)
                        if len(personNr2_list) == 2:
                            # If length is 2, create a single row with 'siblings' as the connection
                            row_data = {'personNr1': personNr2_list[0], 'personNr2': personNr2_list[1], 'connection': 'siblings'}
                            parent_relationships.extend([row_data])
                                    
                        elif len(personNr2_list) > 2:
                            # If length is greater than 2, create rows for all combinations
                            for pair in combinations(personNr2_list, 2):
                                row_data = {'personNr1': pair[0], 'personNr2': pair[1], 'connection': 'siblings'}
                                #dfs_to_concat.extend([pd.DataFrame(row_data)])
                                parent_relationships.extend([row_data])


    if not parent_relationships:
        parent_relationships = []

            
    return parent_relationships

In [398]:
data = {'personNr1': [2, 2, 4],
        'personNr2': [1, 1, 5],
        'connection': ['parent of', 'partner of', 'partner of']}

data1 = {'PersonNr': [2, 1, 4],
        'Alder': [18, 19, 60]}
connections = [{'personNr1': p1, 'personNr2': p2, 'connection': conn} for p1, p2, conn in zip(data['personNr1'], data['personNr2'], data['connection'])]
print(connections)
print(pd.DataFrame(connections))

print(find_parent_child_relationship(pd.DataFrame(data1), 2004))

[{'personNr1': 2, 'personNr2': 1, 'connection': 'parent of'}, {'personNr1': 2, 'personNr2': 1, 'connection': 'partner of'}, {'personNr1': 4, 'personNr2': 5, 'connection': 'partner of'}]
   personNr1  personNr2  connection
0          2          1   parent of
1          2          1  partner of
2          4          5  partner of


KeyError: 'Barn11_15'

In [442]:
def find_relationships_grouped(group):
    couple_relationships = pd.DataFrame()

    # Assuming the DataFrame is already sorted by 'PersonNr' within each group
    selected = group[['PersonNr', 'Alder']]

    for i in range(len(selected) - 1):
        age_difference = abs(selected.iloc[i, 1] - selected.iloc[i+1, 1])

        if age_difference <= 13:
            relationship_row = pd.DataFrame({
                'personNr1': selected.iloc[i, 0],
                'personNr2': selected.iloc[i+1, 0],
                'connection': "partner of"
            }, index=[0])

            couple_relationships = pd.concat([couple_relationships, relationship_row], ignore_index=True)

    return couple_relationships

In [445]:
num_rows = 100
data = {
    'group_col': np.random.choice(['A', 'B', 'C'], num_rows),
    'PersonNr': np.arange(1, num_rows + 1),
    'Alder': np.random.randint(18, 60, num_rows)
}
df = pd.DataFrame(data)
grouped_df = df.groupby('group_col')

result_per_group = grouped_df.apply(find_relationships_grouped)
# Concatenate the lists into a single list
result_df = grouped_df.apply(find_relationships_grouped).reset_index(drop=True)
print(result_df)

# Convert the list of dictionaries to a DataFrame

#print(result_df)

    personNr1  personNr2  connection
0          20         27  partner of
1          27         28  partner of
2          28         29  partner of
3          38         39  partner of
4          43         50  partner of
5          56         57  partner of
6          65         66  partner of
7          66         69  partner of
8          69         71  partner of
9          75         77  partner of
10         77         80  partner of
11         80         82  partner of
12         83         84  partner of
13         95         96  partner of
14         96         99  partner of
15         99        100  partner of
16          4          8  partner of
17         14         21  partner of
18         21         24  partner of
19         24         30  partner of
20         30         31  partner of
21         33         37  partner of
22         41         44  partner of
23         51         55  partner of
24         55         63  partner of
25         63         64  partner of
2